In [6]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [7]:
import os

from git import Repo

In [8]:
repo_path = "./test_repo"

In [9]:
repo = Repo.clone_from("https://github.com/ViniciusPonte/devstore-ignite",  to_path=repo_path)

In [10]:
loader = GenericLoader.from_filesystem(
    repo_path + "/src/app",
    glob="**/*",
    suffixes=[".tsx"],
    exclude=["**/non-utf-8-encoding.tsx"],
    parser=LanguageParser(language=Language.TS, parser_threshold=500)
)

documents = loader.load()
len(documents)

9

In [11]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.TS, chunk_size = 1500, chunk_overlap = 200
)

texts = python_splitter.split_documents(documents=documents)
len(texts)
print(texts)

[Document(metadata={'source': 'test_repo/src/app/layout.tsx', 'language': <Language.TS: 'ts'>}, page_content='import type { Metadata } from \'next\'\nimport { Inter } from \'next/font/google\'\nimport \'./globals.css\'\n\nconst inter = Inter({ subsets: [\'latin\'], variable: \'--font-inter\' })\n\nexport const metadata: Metadata = {\n  title: {\n    template: \'%s | devstore\',\n    default: \'devstore\',\n  },\n}\n\nexport default function RootLayout({\n  children,\n}: {\n  children: React.ReactNode\n}) {\n  return (\n    <html className={inter.variable} lang="pt">\n      <body className="bg-zinc-950 text-zinc-50 antialiased">{children}</body>\n    </html>\n  )\n}'), Document(metadata={'source': 'test_repo/src/app/(store)/layout.tsx', 'language': <Language.TS: 'ts'>}, page_content='import { Header } from \'@/components/header\'\nimport { CartProvider } from \'@/contexts/cart-context\'\nimport { ReactNode } from \'react\'\n\nexport default function StoreLayout({ children }: { children:

In [12]:
os.environ["OPENAI_API_KEY"] = ""

In [13]:
embedding = OpenAIEmbeddings(disallowed_special=())

db = Chroma.from_documents(texts, embedding=embedding, persist_directory="db_code_texts")

vectordb = Chroma(persist_directory="db_code_texts", embedding_function=embedding)

retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 8})

/var/folders/t9/2wfh8wr96y910pmrrcxplfb00000gn/T/ipykernel_2407/1920532231.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="db_code_texts", embedding_function=embedding)


In [14]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)

/var/folders/t9/2wfh8wr96y910pmrrcxplfb00000gn/T/ipykernel_2407/1082775704.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)


In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseado no contexto fornecido abaixo: \n\n {context}."
        ),
        ("user", "{input}")
    ]
)

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para os componentes desse código?"})

print(response['answer'])

1. **Componente `CurrentSearch`**:
   - O componente parece estar bem estruturado e não há problemas aparentes.
   - Você pode considerar adicionar um tratamento de erro caso `query` seja `null`.

2. **Componente `RootLayout`**:
   - O componente parece estar bem estruturado e não há problemas aparentes.
   - Verifique se a importação de `Inter` está correta e se a variável `inter` está sendo utilizada corretamente.

3. **Função `getProduct`**:
   - A função parece estar bem implementada e não há problemas aparentes.
   - Certifique-se de que a API `/product/${slug}` está retornando os dados esperados.

4. **Função `generateMetadata`**:
   - A função parece estar bem implementada e não há problemas aparentes.
   - Certifique-se de que a função `
